In [3]:
import win32com.client
from datetime import datetime
import pytz  # pip install pytz


def search_outlook_mails(sender=None, subject=None, date_after=None, date_before=None):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    results = []

    def search_folder(folder):
        try:
            # Vérifier si le dossier contient des mails
            if folder.DefaultItemType != 0:  # 0 = MailItem
                return

            # Préparer la restriction de dates
            restrictions = []
            if date_after:
                restrictions.append(f"[ReceivedTime] >= '{date_after.strftime('%m/%d/%Y %I:%M %p')}'")
            if date_before:
                restrictions.append(f"[ReceivedTime] <= '{date_before.strftime('%m/%d/%Y %I:%M %p')}'")

            restriction_query = " AND ".join(restrictions)
            items = folder.Items

            # Appliquer la restriction uniquement si c'est un dossier mail
            if restriction_query:
                items = items.Restrict(restriction_query)

            # Trier les mails par date décroissante
            items.Sort("[ReceivedTime]", True)

            for item in items:
                if item.Class == 43:  # MailItem uniquement
                    match = True
                    if sender and sender.lower() not in (item.SenderEmailAddress or "").lower():
                        match = False
                    if subject and subject.lower() not in (item.Subject or "").lower():
                        match = False

                    if match:
                        results.append({
                            "Sender": item.SenderName,
                            "Email": item.SenderEmailAddress,
                            "Subject": item.Subject,
                            "Date": item.ReceivedTime.strftime("%Y-%m-%d %H:%M:%S"),
                            "Folder": folder.Name
                        })

            # Recherche récursive dans les sous-dossiers
            for subfolder in folder.Folders:
                search_folder(subfolder)

        except Exception as e:
            print(f"⚠️ Erreur dans le dossier {folder.Name}: {e}")

    # Parcourir toutes les boîtes et archives
    for store in outlook.Folders:
        search_folder(store)

    return results


if __name__ == "__main__":
    # Définir fuseau cohérent
    tz = pytz.timezone("UTC")
    date_after = tz.localize(datetime(2025, 7, 1))
    date_before = tz.localize(datetime(2025, 8, 10))

    mails = search_outlook_mails(
        sender="compliance_eme@gos.orange.com",
        subject="[VITIB][EME][Production][CF01]: Liste alertes kyc de la periode",
        date_after=date_after,
        date_before=date_before
    )

    print(f"\n--- {len(mails)} mails trouvés ---")
    for mail in mails:
        print(f"[{mail['Date']}] {mail['Sender']} ({mail['Email']}) - {mail['Subject']} (Dossier: {mail['Folder']})")


⚠️ Erreur dans le dossier Boîte de réception: (-2147467262, 'Cette interface n’est pas prise en charge', None, None)

--- 17 mails trouvés ---
[2025-02-01 06:10:16] compliance_eme@gos.orange.com (compliance_eme@gos.orange.com) - [VITIB][EME][Production][CF01]: Liste alertes kyc de la periode (Dossier: Boîte de réception)
[2025-06-01 08:10:09] compliance_eme@gos.orange.com (compliance_eme@gos.orange.com) - [VITIB][EME][Production][CF01]: Liste alertes kyc de la periode (Dossier: Boîte de réception)
[2025-06-01 07:10:07] compliance_eme@gos.orange.com (compliance_eme@gos.orange.com) - [VITIB][EME][Production][CF01]: Liste alertes kyc de la periode (Dossier: Boîte de réception)
[2025-08-01 17:10:11] compliance_eme@gos.orange.com (compliance_eme@gos.orange.com) - [VITIB][EME][Production][CF01]: Liste alertes kyc de la periode (Dossier: Boîte de réception)
[2025-08-01 16:10:08] compliance_eme@gos.orange.com (compliance_eme@gos.orange.com) - [VITIB][EME][Production][CF01]: Liste alertes kyc d